In [3]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from pandas import read_csv
import os


Election results from OpenElections (http://openelections.net/)
Shapefile from the Utah Automated Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/)

Several transcription errors were corrected from OpenElections using their raw results gathered from counties (https://github.com/openelections/openelections-sources-ut/tree/master/2018%20General%20Election). OE results in Rich and Morgan Counties were based on non-final reports, which were collected from county sources.

Emery County - 8 "canvas votes" were distributed to precincts.

Weber County - 6 precincts had their results suppressed, since all cast fewer than 15 votes, for a total of 42 votes. These were distributed based on the number of votes cast and the reported vote in 2016, which was not suppressed.

G18USSRROM - Mitt Romney (Republican Party)
G18USSDWIL - Jenny Wilson (Democratic Party)
G18USSIMCC - Reed C. McCandless (Independent American Party)
G18USSLBOW - Craig R. Bowden (Libertarian Party)
G18USSCAAL - Tim Aalders (Conservative Party)
G18USSOWRI - Write-in Votes

In [4]:
ut_election_results = pd.read_csv('raw-from-source/20181106__ut__general__precinct.csv')
ut_vest = gp.read_file("raw-from-source/ut_2018/ut_2018.shp")
ut_geo = gp.read_file("raw-from-source/Utah_Vista_Ballot_Areas/VistaBallotAreas.shp")

In [102]:
pd.set_option('display.max_rows', 10)
display(ut_election_results)
print(ut_election_results['county'].unique())
ut_senators = ut_election_results[ut_election_results['office'] == "U.S. Senate"]
display(ut_senators)

,county,precinct,office,district,candidate,party,votes
0,Wayne,Fremont,U.S. Senate,NaN,Tim Aalders,Con,7
1,Wayne,Loa in/out,U.S. Senate,NaN,Tim Aalders,Con,6
2,Wayne,Lyman in/out,U.S. Senate,NaN,Tim Aalders,Con,6
3,Wayne,Bicknell in/out,U.S. Senate,NaN,Tim Aalders,Con,8
4,Wayne,Teesdale,U.S. Senate,NaN,Tim Aalders,Con,3
...,...,...,...,...,...,...,...
41993,Morgan,Croydon 15.5,U S House,NaN,Rob Bishop,REP,64
41994,Morgan,Croydon 15.5,U S House,NaN,Eric Eliason,UUP,8
41995,Morgan,Croydon 15.5,U S House,NaN,Adam Davis,GRN,2
41996,Morgan,Croydon 15.5,Utah State House,53.0,Logan Wilde,REP,80


['Wayne' 'Salt Lake' 'Wasatch' 'Iron' 'Beaver' 'Utah' 'Carbon' 'Juab'
 'Kane' 'Garfield' 'Daggett' 'Sanpete' 'Sevier' 'Box Elder' 'Grand'
 'Millard' 'San Juan' 'Tooele' 'Duchesne' 'Cache' 'Weber' 'Davis' 'Summit'
 'Uintah' 'Emery' 'Piute' 'Washington' 'Rich' 'Morgan']


,county,precinct,office,district,candidate,party,votes
41644,Morgan,Mountain Green 1.1,Registered Voters,NaN,NaN,NaN,536
41645,Morgan,Mountain Green 1.1,Ballots Cast,NaN,NaN,NaN,427
41646,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,CON,1
41647,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,UUP,0
41648,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,LIB,1
...,...,...,...,...,...,...,...
41993,Morgan,Croydon 15.5,U S House,NaN,Rob Bishop,REP,64
41994,Morgan,Croydon 15.5,U S House,NaN,Eric Eliason,UUP,8
41995,Morgan,Croydon 15.5,U S House,NaN,Adam Davis,GRN,2
41996,Morgan,Croydon 15.5,Utah State House,53.0,Logan Wilde,REP,80


,county,precinct,office,district,candidate,party,votes
0,Wayne,Fremont,U.S. Senate,NaN,Tim Aalders,Con,7
1,Wayne,Loa in/out,U.S. Senate,NaN,Tim Aalders,Con,6
2,Wayne,Lyman in/out,U.S. Senate,NaN,Tim Aalders,Con,6
3,Wayne,Bicknell in/out,U.S. Senate,NaN,Tim Aalders,Con,8
4,Wayne,Teesdale,U.S. Senate,NaN,Tim Aalders,Con,3
...,...,...,...,...,...,...,...
41609,Rich,1,U.S. Senate,NaN,Write-ins,NaN,0
41610,Rich,2,U.S. Senate,NaN,Write-ins,NaN,0
41611,Rich,3,U.S. Senate,NaN,Write-ins,NaN,0
41612,Rich,4,U.S. Senate,NaN,Write-ins,NaN,0


In [12]:
sos_beaver = ut_senators[ut_senators['county'] == "Beaver"]
display(sos_beaver)

,county,precinct,office,district,candidate,party,votes
17721,Beaver,Beaver 1,U.S. Senate,NaN,Tim Aalders,Con,19
17722,Beaver,Beaver 2,U.S. Senate,NaN,Tim Aalders,Con,17
17723,Beaver,Beaver 3,U.S. Senate,NaN,Tim Aalders,Con,12
17724,Beaver,Beaver 4,U.S. Senate,NaN,Tim Aalders,Con,23
17725,Beaver,Greenville,U.S. Senate,NaN,Tim Aalders,Con,1
17726,Beaver,Milford 1,U.S. Senate,NaN,Tim Aalders,Con,0
17727,Beaver,Milford 2,U.S. Senate,NaN,Tim Aalders,Con,18
17728,Beaver,Milford 3,U.S. Senate,NaN,Tim Aalders,Con,8
17729,Beaver,Minersville,U.S. Senate,NaN,Tim Aalders,Con,40
17730,Beaver,Beaver 1,U.S. Senate,NaN,Craig Bowden,Lbt,9


In [13]:
vest_beaver = ut_vest[ut_vest['CountyID'] == 1]
display(vest_beaver)

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
5,1,ML01,1-ML01,38,23,0,6,0,0,"POLYGON ((324513.800 4253004.500, 324578.800 4..."
6,1,ML02,1-ML02,182,102,3,7,18,3,"POLYGON ((322814.500 4251988.700, 322875.200 4..."
7,1,ML03,1-ML03,61,16,1,3,8,1,"POLYGON ((344948.320 4270854.760, 344950.160 4..."
8,1,MV01,1-MV01,243,19,8,3,40,1,"POLYGON ((328777.760 4237537.600, 329104.140 4..."


In [16]:
## box elder
sos_box_elder = ut_senators[ut_senators['county'] == "Box Elder"]
print(sos_box_elder['precinct'].unique())

['Bear River East' 'Bear River' 'Beaver Dam/Collinston' 'Brigham City 01'
 'Brigham City 02' 'Brigham City 03' 'Brigham City 04' 'Brigham City 05'
 'Brigham City 06' 'Brigham City 07' 'Brigham City 08' 'Brigham City 09'
 'Brigham City 10' 'Brigham City 11' 'Brigham City 12' 'Brigham City 13'
 'Brigham City 14' 'Brigham City 15' 'Brigham City North' 'Clear Creek'
 'Corinne 1' 'Corinne 2' 'Deweyville' 'Elwood' 'Fielding 1' 'Fielding 2'
 'Garland 1' 'Garland 2' 'Garland East' 'Grouse Creek' 'Honeyville'
 'Howell' 'Mantua' 'Park Valley' 'Perry 1' 'Perry 2' 'Perry 3' 'Perry 4'
 'Plymouth' 'Portage' 'Promontory' 'Riverside' 'Snowville' 'South Willard'
 'Tremonton 1' 'Tremonton 2' 'Tremonton 3' 'Tremonton 4' 'Tremonton 5'
 'Tremonton South' 'Tremonton West' 'Yost']


In [17]:
vest_box_elder = ut_vest[ut_vest['CountyID'] == 2]
display(vest_box_elder)

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
9,2,BC01,2-BC01,432,113,16,20,20,2,"POLYGON ((414524.370 4593984.900, 414522.580 4..."
10,2,BC02,2-BC02,380,61,4,8,14,1,"POLYGON ((414555.830 4595123.510, 414524.370 4..."
11,2,BC03,2-BC03,306,104,7,23,26,5,"POLYGON ((415201.410 4594509.040, 415185.980 4..."
12,2,BC04,2-BC04,296,85,8,21,18,3,"POLYGON ((415237.300 4595917.200, 415225.500 4..."
13,2,BC05,2-BC05,215,92,8,14,14,3,"MULTIPOLYGON (((415245.680 4596194.430, 415237..."
14,2,BC06,2-BC06,428,61,11,16,24,5,"POLYGON ((411603.260 4601803.670, 411600.700 4..."
15,2,BC07,2-BC07,344,83,11,17,12,2,"POLYGON ((415311.780 4598727.060, 415296.070 4..."
16,2,BC08,2-BC08,175,58,6,9,7,1,"POLYGON ((415664.790 4597477.770, 415634.900 4..."
17,2,BC09,2-BC09,228,38,5,6,4,0,"POLYGON ((416035.600 4596274.800, 416036.550 4..."
18,2,BC10,2-BC10,361,67,5,8,7,0,"POLYGON ((416713.460 4597882.240, 419201.140 4..."


In [39]:
print(ut_vest['CountyID'].unique())


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29]


In [41]:
ut_vest_2016 = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")
ut_dict = ut_vest_2016.set_index('precinct')['PrecinctID'].to_dict()
print(ut_dict)
## split dict by counties, beaver ok box elder no -> need to add filler 0's to get right code

{'Beaver 1': 'BV01', 'Beaver 2': 'BV02', 'Beaver 3': 'BV03', 'Beaver 4': 'BV04', 'Greenville': 'GR01', 'Milford 1': 'ML01', 'Milford 2': 'ML02', 'Milford 3': 'ML03', 'Minersville': 'MV01', 'Brigham City 01': 'BC01', 'Brigham City 02': 'BC02', 'Brigham City 03': 'BC03', 'Brigham City 04': 'BC04', 'Brigham City 05': 'BC05', 'Brigham City 06': 'BC06', 'Brigham City 07': 'BC07', 'Brigham City 08': 'BC08', 'Brigham City 09': 'BC09', 'Brigham City 10': 'BC10', 'Brigham City 11': 'BC11', 'Brigham City 12': 'BC12', 'Brigham City 13': 'BC13', 'Brigham City 14': 'BC14', 'Brigham City 15': 'BC15', 'Brigham City North': 'BCN', 'Beaver Dam/Collinston': 'BD/C', 'Bear River': 'BEAR', 'Bear River East': 'BRE', 'Clear Creek': 'CC', 'Corinne 1': 'COR1', 'Corinne 2': 'COR2', 'Deweyville': 'DEWY', 'Elwood': 'ELWD', 'Fielding 01': 'FLDG1', 'Fielding 02': 'FLDG2', 'Garland 01': 'GAR1', 'Garland 02': 'GAR2', 'Garland East': 'GARE', 'Grouse Creek': 'GC', 'Honeyville': 'HONY', 'Howell': 'HOWL', 'Mantua': 'MANT

In [23]:
display(ut_senators)

,county,precinct,office,district,candidate,party,votes
0,Wayne,Fremont,U.S. Senate,NaN,Tim Aalders,Con,7
1,Wayne,Loa in/out,U.S. Senate,NaN,Tim Aalders,Con,6
2,Wayne,Lyman in/out,U.S. Senate,NaN,Tim Aalders,Con,6
3,Wayne,Bicknell in/out,U.S. Senate,NaN,Tim Aalders,Con,8
4,Wayne,Teesdale,U.S. Senate,NaN,Tim Aalders,Con,3
...,...,...,...,...,...,...,...
41609,Rich,1,U.S. Senate,NaN,Write-ins,NaN,0
41610,Rich,2,U.S. Senate,NaN,Write-ins,NaN,0
41611,Rich,3,U.S. Senate,NaN,Write-ins,NaN,0
41612,Rich,4,U.S. Senate,NaN,Write-ins,NaN,0


In [34]:
ut_senators['mergeid'] = ut_senators['precinct'].map(ut_dict)


<ipython-input-34-db895ea29996>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ut_senators['mergeid'] = ut_senators['precinct'].map(ut_dict)


,county,precinct,office,district,candidate,party,votes,mergeid
23615,Box Elder,Bear River East,U.S. Senate,NaN,TIM AALDERS,CON,13,BRE
23616,Box Elder,Bear River East,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,6,BRE
23617,Box Elder,Bear River East,U.S. Senate,NaN,REED C. McCANDLESS,IAP,6,BRE
23618,Box Elder,Bear River East,U.S. Senate,NaN,JENNY WILSON,DEM,31,BRE
23619,Box Elder,Bear River East,U.S. Senate,NaN,MITT ROMNEY,REP,159,BRE
...,...,...,...,...,...,...,...,...
24369,Box Elder,Yost,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,2,YOST
24370,Box Elder,Yost,U.S. Senate,NaN,REED C. McCANDLESS,IAP,0,YOST
24371,Box Elder,Yost,U.S. Senate,NaN,JENNY WILSON,DEM,0,YOST
24372,Box Elder,Yost,U.S. Senate,NaN,MITT ROMNEY,REP,13,YOST


In [97]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
dict2 = ut_senators[ut_senators['mergeid'].isnull()]['county'].unique() 
for county in ut_senators['county'].unique():
    if county not in dict2:
        print(county)
print(ut_senators['county'].unique())
print(len(ut_senators['county'].unique()))

display(ut_senators[ut_senators['county']=='Morgan'])
display(ut_senators[ut_senators['county']=='Rich'])

Beaver
Duchesne
['Wayne' 'Salt Lake' 'Wasatch' 'Iron' 'Beaver' 'Utah' 'Carbon' 'Juab'
 'Kane' 'Garfield' 'Daggett' 'Sanpete' 'Sevier' 'Box Elder' 'Grand'
 'Millard' 'San Juan' 'Tooele' 'Duchesne' 'Cache' 'Weber' 'Davis' 'Summit'
 'Uintah' 'Emery' 'Piute' 'Washington' 'Rich']
28


,county,precinct,office,district,candidate,party,votes,mergeid


,county,precinct,office,district,candidate,party,votes,mergeid
41584,Rich,1,U.S. Senate,NaN,Tim Aalders,Con,6,NaN
41585,Rich,2,U.S. Senate,NaN,Tim Aalders,Con,9,NaN
41586,Rich,3,U.S. Senate,NaN,Tim Aalders,Con,5,NaN
41587,Rich,4,U.S. Senate,NaN,Tim Aalders,Con,8,NaN
41588,Rich,5,U.S. Senate,NaN,Tim Aalders,Con,5,NaN
41589,Rich,1,U.S. Senate,NaN,Craig Bowden,Lbt,4,NaN
41590,Rich,2,U.S. Senate,NaN,Craig Bowden,Lbt,1,NaN
41591,Rich,3,U.S. Senate,NaN,Craig Bowden,Lbt,12,NaN
41592,Rich,4,U.S. Senate,NaN,Craig Bowden,Lbt,2,NaN
41593,Rich,5,U.S. Senate,NaN,Craig Bowden,Lbt,5,NaN


In [99]:
display(ut_vest[ut_vest['CountyID'] == 15])
ut_vest_mo = ut_vest[ut_vest['CountyID'] == 15]
ut_vest_mo['sum'] = ut_vest_mo['G18USSRROM'] + ut_vest_mo['G18USSDWIL'] + ut_vest_mo['G18USSIMCC'] + ut_vest_mo['G18USSLBOW'] + ut_vest_mo['G18USSCAAL'] + ut_vest_mo['G18USSOWRI']
display(ut_vest_mo)
display(ut_vest[ut_vest['CountyID'] == 1])

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
568,15,1:1,15-1:1,307,79,5,10,20,3,"POLYGON ((431569.630 4561834.170, 431550.130 4..."
569,15,10:4,15-10:4,319,43,0,18,11,1,"POLYGON ((440762.270 4544845.210, 440793.230 4..."
570,15,11:4,15-11:4,148,37,4,8,9,0,"POLYGON ((443328.250 4542698.210, 443346.490 4..."
571,15,12:4,15-12:4,301,45,3,9,9,0,"POLYGON ((443346.490 4542648.920, 443444.380 4..."
572,15,13:5,15-13:5,444,39,8,6,15,3,"POLYGON ((445082.810 4542456.070, 444933.350 4..."
573,15,14:5,15-14:5,304,45,7,5,14,0,"POLYGON ((453333.350 4542408.850, 447902.690 4..."
574,15,15:5,15-15:5,65,6,0,4,11,0,"POLYGON ((465272.500 4547052.500, 465236.000 4..."
575,15,2:1,15-2:1,375,70,1,6,11,0,"POLYGON ((433919.490 4563251.000, 433937.490 4..."
576,15,3:1,15-3:1,239,28,4,7,15,5,"POLYGON ((439816.160 4559822.610, 439813.120 4..."
577,15,4:2,15-4:2,509,52,2,10,10,0,"POLYGON ((441169.530 4557696.280, 441165.060 4..."


/Users/izzy/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry,sum
568,15,1:1,15-1:1,307,79,5,10,20,3,"POLYGON ((431569.630 4561834.170, 431550.130 4...",424
569,15,10:4,15-10:4,319,43,0,18,11,1,"POLYGON ((440762.270 4544845.210, 440793.230 4...",392
570,15,11:4,15-11:4,148,37,4,8,9,0,"POLYGON ((443328.250 4542698.210, 443346.490 4...",206
571,15,12:4,15-12:4,301,45,3,9,9,0,"POLYGON ((443346.490 4542648.920, 443444.380 4...",367
572,15,13:5,15-13:5,444,39,8,6,15,3,"POLYGON ((445082.810 4542456.070, 444933.350 4...",515
573,15,14:5,15-14:5,304,45,7,5,14,0,"POLYGON ((453333.350 4542408.850, 447902.690 4...",375
574,15,15:5,15-15:5,65,6,0,4,11,0,"POLYGON ((465272.500 4547052.500, 465236.000 4...",86
575,15,2:1,15-2:1,375,70,1,6,11,0,"POLYGON ((433919.490 4563251.000, 433937.490 4...",463
576,15,3:1,15-3:1,239,28,4,7,15,5,"POLYGON ((439816.160 4559822.610, 439813.120 4...",298
577,15,4:2,15-4:2,509,52,2,10,10,0,"POLYGON ((441169.530 4557696.280, 441165.060 4...",583


,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
5,1,ML01,1-ML01,38,23,0,6,0,0,"POLYGON ((324513.800 4253004.500, 324578.800 4..."
6,1,ML02,1-ML02,182,102,3,7,18,3,"POLYGON ((322814.500 4251988.700, 322875.200 4..."
7,1,ML03,1-ML03,61,16,1,3,8,1,"POLYGON ((344948.320 4270854.760, 344950.160 4..."
8,1,MV01,1-MV01,243,19,8,3,40,1,"POLYGON ((328777.760 4237537.600, 329104.140 4..."


In [106]:
pd.set_option('display.max_rows', 500)

In [105]:
display(ut_election_results[ut_election_results['county'] == 'Morgan'])
ut_morgan = ut_election_results[ut_election_results['county'] == 'Morgan']


,county,precinct,office,district,candidate,party,votes
41644,Morgan,Mountain Green 1.1,Registered Voters,NaN,NaN,NaN,536
41645,Morgan,Mountain Green 1.1,Ballots Cast,NaN,NaN,NaN,427
41646,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,CON,1
41647,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,UUP,0
41648,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,LIB,1
...,...,...,...,...,...,...,...
41993,Morgan,Croydon 15.5,U S House,NaN,Rob Bishop,REP,64
41994,Morgan,Croydon 15.5,U S House,NaN,Eric Eliason,UUP,8
41995,Morgan,Croydon 15.5,U S House,NaN,Adam Davis,GRN,2
41996,Morgan,Croydon 15.5,Utah State House,53.0,Logan Wilde,REP,80


,county,precinct,office,district,candidate,party,votes
41659,Morgan,Mountain Green 1.1,U S House,NaN,Lee Castillo,DEM,75
41660,Morgan,Mountain Green 1.1,U S House,NaN,Rob Bishop,REP,280
41661,Morgan,Mountain Green 1.1,U S House,NaN,Eric Eliason,UUP,53
41662,Morgan,Mountain Green 1.1,U S House,NaN,Adam Davis,GRN,8
41683,Morgan,Mountain Green 2.1,U S House,NaN,Lee Castillo,DEM,72
...,...,...,...,...,...,...,...
41971,Morgan,No Mogan 14.5,U S House,NaN,Adam Davis,GRN,7
41992,Morgan,Croydon 15.5,U S House,NaN,Lee Castillo,DEM,9
41993,Morgan,Croydon 15.5,U S House,NaN,Rob Bishop,REP,64
41994,Morgan,Croydon 15.5,U S House,NaN,Eric Eliason,UUP,8


In [109]:
display(ut_morgan[ut_morgan['office'] == 'U S Senate'])

,county,precinct,office,district,candidate,party,votes
41653,Morgan,Mountain Green 1.1,U S Senate,NaN,Tim Aalders,CON,20
41654,Morgan,Mountain Green 1.1,U S Senate,NaN,Craig R. Bowden,LIB,10
41655,Morgan,Mountain Green 1.1,U S Senate,NaN,Reed C. McCandless,IAP,5
41656,Morgan,Mountain Green 1.1,U S Senate,NaN,Jenny Wilson,DEM,79
41657,Morgan,Mountain Green 1.1,U S Senate,NaN,Mitt Romney,REP,307
41658,Morgan,Mountain Green 1.1,U S Senate,NaN,Write-In Votes,NaN,3
41677,Morgan,Mountain Green 2.1,U S Senate,NaN,Tim Aalders,CON,11
41678,Morgan,Mountain Green 2.1,U S Senate,NaN,Craig R. Bowden,LIB,6
41679,Morgan,Mountain Green 2.1,U S Senate,NaN,Reed C. McCandless,IAP,1
41680,Morgan,Mountain Green 2.1,U S Senate,NaN,Jenny Wilson,DEM,70


In [108]:
display(ut_election_results[ut_election_results['precinct'] == 'Mountain Green 1.1'])

,county,precinct,office,district,candidate,party,votes
41644,Morgan,Mountain Green 1.1,Registered Voters,NaN,NaN,NaN,536
41645,Morgan,Mountain Green 1.1,Ballots Cast,NaN,NaN,NaN,427
41646,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,CON,1
41647,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,UUP,0
41648,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,LIB,1
41649,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,REP,85
41650,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,GRN,0
41651,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,DEM,18
41652,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,IAP,9
41653,Morgan,Mountain Green 1.1,U S Senate,NaN,Tim Aalders,CON,20


In [60]:
# Beaver
ut_beaver_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == 1]
ut_dict1 = ut_beaver_2016.set_index('precinct')['PrecinctID'].to_dict()
print(ut_dict1)

{'Beaver 1': 'BV01', 'Beaver 2': 'BV02', 'Beaver 3': 'BV03', 'Beaver 4': 'BV04', 'Greenville': 'GR01', 'Milford 1': 'ML01', 'Milford 2': 'ML02', 'Milford 3': 'ML03', 'Minersville': 'MV01'}


In [94]:
# Box Elder County
ut_be_2016 = ut_vest_2016[ut_vest_2016['CountyID'] == 2]
ut_dict2 = ut_be_2016.set_index('precinct')['PrecinctID'].to_dict()
print(ut_dict2)
print(len(ut_dict2))
ut_dict2['Tremonton 04'] = ut_dict2.pop('Trementon 04')

{'Brigham City 01': 'BC01', 'Brigham City 02': 'BC02', 'Brigham City 03': 'BC03', 'Brigham City 04': 'BC04', 'Brigham City 05': 'BC05', 'Brigham City 06': 'BC06', 'Brigham City 07': 'BC07', 'Brigham City 08': 'BC08', 'Brigham City 09': 'BC09', 'Brigham City 10': 'BC10', 'Brigham City 11': 'BC11', 'Brigham City 12': 'BC12', 'Brigham City 13': 'BC13', 'Brigham City 14': 'BC14', 'Brigham City 15': 'BC15', 'Brigham City North': 'BCN', 'Beaver Dam/Collinston': 'BD/C', 'Bear River': 'BEAR', 'Bear River East': 'BRE', 'Clear Creek': 'CC', 'Corinne 1': 'COR1', 'Corinne 2': 'COR2', 'Deweyville': 'DEWY', 'Elwood': 'ELWD', 'Fielding 01': 'FLDG1', 'Fielding 02': 'FLDG2', 'Garland 01': 'GAR1', 'Garland 02': 'GAR2', 'Garland East': 'GARE', 'Grouse Creek': 'GC', 'Honeyville': 'HONY', 'Howell': 'HOWL', 'Mantua': 'MANT', 'Park Valley': 'PARK', 'Perry 01': 'PER1', 'Perry 02': 'PER2', 'Perry 03': 'PER3', 'Perry 04': 'PER4', 'Plymouth': 'PLYM', 'Portage': 'PORT', 'Promontory': 'PROM', 'Riverside': 'RIV', '

In [95]:
be = (ut_senators[ut_senators['county'] == 'Box Elder'])
addzero = (be[be['mergeid'].isnull()]['precinct'].unique())
print(addzero)
for precinct in addzero:
    print(precinct)
    new_name = precinct[:-1] + '0' + precinct[-1]
    print(new_name)
    ut_dict2[precinct] = ut_dict2.pop(new_name)
    
print(ut_dict2)

['Fielding 1' 'Fielding 2' 'Garland 1' 'Garland 2' 'Perry 1' 'Perry 2'
 'Perry 3' 'Perry 4' 'Tremonton 1' 'Tremonton 2' 'Tremonton 3'
 'Tremonton 4' 'Tremonton 5']
Fielding 1
Fielding 01
Fielding 2
Fielding 02
Garland 1
Garland 01
Garland 2
Garland 02
Perry 1
Perry 01
Perry 2
Perry 02
Perry 3
Perry 03
Perry 4
Perry 04
Tremonton 1
Tremonton 01
Tremonton 2
Tremonton 02
Tremonton 3
Tremonton 03
Tremonton 4
Tremonton 04
Tremonton 5
Tremonton 05
{'Brigham City 01': 'BC01', 'Brigham City 02': 'BC02', 'Brigham City 03': 'BC03', 'Brigham City 04': 'BC04', 'Brigham City 05': 'BC05', 'Brigham City 06': 'BC06', 'Brigham City 07': 'BC07', 'Brigham City 08': 'BC08', 'Brigham City 09': 'BC09', 'Brigham City 10': 'BC10', 'Brigham City 11': 'BC11', 'Brigham City 12': 'BC12', 'Brigham City 13': 'BC13', 'Brigham City 14': 'BC14', 'Brigham City 15': 'BC15', 'Brigham City North': 'BCN', 'Beaver Dam/Collinston': 'BD/C', 'Bear River': 'BEAR', 'Bear River East': 'BRE', 'Clear Creek': 'CC', 'Corinne 1': 'COR1

In [64]:
print(ut_senators[ut_senators['county'] == 'Box Elder']['precinct'].unique())

['Bear River East' 'Bear River' 'Beaver Dam/Collinston' 'Brigham City 01'
 'Brigham City 02' 'Brigham City 03' 'Brigham City 04' 'Brigham City 05'
 'Brigham City 06' 'Brigham City 07' 'Brigham City 08' 'Brigham City 09'
 'Brigham City 10' 'Brigham City 11' 'Brigham City 12' 'Brigham City 13'
 'Brigham City 14' 'Brigham City 15' 'Brigham City North' 'Clear Creek'
 'Corinne 1' 'Corinne 2' 'Deweyville' 'Elwood' 'Fielding 1' 'Fielding 2'
 'Garland 1' 'Garland 2' 'Garland East' 'Grouse Creek' 'Honeyville'
 'Howell' 'Mantua' 'Park Valley' 'Perry 1' 'Perry 2' 'Perry 3' 'Perry 4'
 'Plymouth' 'Portage' 'Promontory' 'Riverside' 'Snowville' 'South Willard'
 'Tremonton 1' 'Tremonton 2' 'Tremonton 3' 'Tremonton 4' 'Tremonton 5'
 'Tremonton South' 'Tremonton West' 'Yost']


In [65]:
ut_senators[ut_senators['county'] == 'Box Elder']

,county,precinct,office,district,candidate,party,votes,mergeid
23615,Box Elder,Bear River East,U.S. Senate,NaN,TIM AALDERS,CON,13,BRE
23616,Box Elder,Bear River East,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,6,BRE
23617,Box Elder,Bear River East,U.S. Senate,NaN,REED C. McCANDLESS,IAP,6,BRE
23618,Box Elder,Bear River East,U.S. Senate,NaN,JENNY WILSON,DEM,31,BRE
23619,Box Elder,Bear River East,U.S. Senate,NaN,MITT ROMNEY,REP,159,BRE
...,...,...,...,...,...,...,...,...
24369,Box Elder,Yost,U.S. Senate,NaN,CRAIG R. BOWDEN,LIB,2,YOST
24370,Box Elder,Yost,U.S. Senate,NaN,REED C. McCANDLESS,IAP,0,YOST
24371,Box Elder,Yost,U.S. Senate,NaN,JENNY WILSON,DEM,0,YOST
24372,Box Elder,Yost,U.S. Senate,NaN,MITT ROMNEY,REP,13,YOST


In [66]:
ut_vest

,CountyID,mergeid,uniqueid,G18USSRROM,G18USSDWIL,G18USSIMCC,G18USSLBOW,G18USSCAAL,G18USSOWRI,geometry
0,1,BV01,1-BV01,402,46,8,9,19,1,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,1-BV02,310,46,3,10,17,0,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,1-BV03,244,30,0,2,12,2,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,1-BV04,84,7,1,4,23,0,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,1-GR01,56,9,6,1,1,1,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
...,...,...,...,...,...,...,...,...,...,...
2332,27,COUN06,27-COUN06,0,0,0,0,0,0,"POLYGON ((255057.190 4154374.220, 256665.370 4..."
2333,27,COUN07,27-COUN07,0,0,0,0,0,0,"POLYGON ((284912.790 4113292.470, 284905.170 4..."
2334,27,SGAIR,27-SGAIR,0,0,0,0,0,0,"POLYGON ((277885.930 4102780.480, 277884.100 4..."
2335,29,LAKE,29-LAKE,0,0,0,0,0,0,"POLYGON ((431658.370 4571461.460, 431647.750 4..."


In [122]:
county_names = ut_election_results['county'].unique()
print(county_names)
county_name_dict = {elem : pd.DataFrame for elem in county_names}
for key in county_name_dict.keys():
    county_name_dict[key] = ut_election_results[:][ut_election_results.county == key]

['Wayne' 'Salt Lake' 'Wasatch' 'Iron' 'Beaver' 'Utah' 'Carbon' 'Juab'
 'Kane' 'Garfield' 'Daggett' 'Sanpete' 'Sevier' 'Box Elder' 'Grand'
 'Millard' 'San Juan' 'Tooele' 'Duchesne' 'Cache' 'Weber' 'Davis' 'Summit'
 'Uintah' 'Emery' 'Piute' 'Washington' 'Rich' 'Morgan']


In [123]:
display(county_name_dict['Morgan'])

,county,precinct,office,district,candidate,party,votes
41644,Morgan,Mountain Green 1.1,Registered Voters,NaN,NaN,NaN,536
41645,Morgan,Mountain Green 1.1,Ballots Cast,NaN,NaN,NaN,427
41646,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,CON,1
41647,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,UUP,0
41648,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,LIB,1
41649,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,REP,85
41650,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,GRN,0
41651,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,DEM,18
41652,Morgan,Mountain Green 1.1,Straight Party,NaN,NaN,IAP,9
41653,Morgan,Mountain Green 1.1,U S Senate,NaN,Tim Aalders,CON,20
